In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import loginInfo

In [15]:
def wait_and_click_element(driver, xpath, timeout=10):
    """
    Waits for an element specified by the given XPath to be present on the page,
    and then clicks the element.

    Args:
        driver (WebDriver): The Selenium WebDriver instance.
        xpath (str): The XPath locator for the element to be clicked.
        timeout (int, optional): The maximum time to wait for the element to be present, in seconds. Default is 10 seconds.

    Example Usage:
    wait_and_click_element(driver, "/html/body/app-component/nav-component/div[2]/div[2]/button[10]", max_wait_time=100)
    """

    wait = WebDriverWait(driver, timeout)
    element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    element.click()

def wait_and_type_element(driver, xpath, text, timeout=10):
    """
    Waits for an element specified by the given XPath to be present on the page,
    and then clicks the element.

    Args:
        driver (WebDriver): The Selenium WebDriver instance.
        xpath (str): The XPath locator for the element to be typed in.
        text (str): The string to be typed in the textbox.
        timeout (int, optional): The maximum time to wait for the element to be present, in seconds. Default is 10 seconds.

    Example Usage:
    wait_and_click_element(driver, "/html/body/app-component/nav-component/div[2]/div[2]/button[10]", "This will be typed", max_wait_time=100)
    """

    wait = WebDriverWait(driver, timeout)
    element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    element.send_keys(text)

In [25]:

def find_inventory_element_and_click(driver, source, timeout = 100):
    """
    Finds the element in the inventory page, and then clicks on the element

    Args:
        driver (WebDriver): The Selenium WebDriver instance.
        source (str): The source of the image to be located. e.g. ".//img[@src='/assets/items/gem-ruby.png']"
        timeout (int, optional): The maximum time to wait for the element to be present, in seconds. Default is 10 seconds.

    Example Usage: 
    find_inventory_element_and_click(driver, source = ".//img[@src='/assets/items/gem-ruby.png']")
    
    """
    # Assuming the webpage is already loaded
    base_xpath = "/html/body/app-component/div/div/div/inventory-page/div/div[1]/div/div[3]/button["
    found = False

    for i in range(1, timeout + 1):
        # Construct the full XPath for each button
        full_xpath = base_xpath + str(i) + "]"

        try:
            # Find the button element
            element = driver.find_element(By.XPATH, full_xpath)

            # Check if the button contains the Ruby image
            image = element.find_element(By.XPATH, source)
            element.click()
        except NoSuchElementException:
            filler = 1+1
        else:
            found = True

        # Stop after max_items even if desired item not found
        if i == timeout or found == True:
            break

def find_inventory_elements(driver, timeout=10):
    """
    Finds all inventory elements and returns them in an array with the name, XPath, and quantity.

    Args:
    driver (WebDriver): The Selenium WebDriver instance.
    timeout (int, optional): The maximum time to wait for elements to be present, in seconds. Default is 10.

    Returns:
    list: A list of dictionaries, each containing 'name', 'xpath', and 'quantity' for an inventory item.

    Example Usage:
    inventory = find_inventory_elements(driver)
    """
    inventory_items = []
    base_xpath = "//button[contains(@class, 'item') and contains(@class, 'ng-star-inserted')]"

    try:
        # Wait for the inventory items to be present
        WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((By.XPATH, base_xpath))
        )
        
        # Find all inventory items
        items = driver.find_elements(By.XPATH, base_xpath)
        

        for i, item in enumerate(items, 1):
            try:
                # Find the image element within the button
                img_element = item.find_element(By.XPATH, ".//img[@_ngcontent-okx-c44]")
                
                # Extract the item name from the image source
                src = img_element.get_attribute("src")
                item_name = src.split("/")[-1].split(".")[0]
                
                # Find the quantity element
                quantity_element = item.find_element(By.XPATH, ".//div[contains(@class, 'amount')]")
                quantity = quantity_element.text
                
                # Add the item to our inventory list
                inventory_items.append({
                    "name": item_name,
                    "xpath": f"({base_xpath})[{i}]",
                    "quantity": quantity
                })
                
            except NoSuchElementException as e:
                print(f"Error processing item {i}: {str(e)}")

    except TimeoutException:
        print(f"Timeout waiting for inventory items to load after {timeout} seconds")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

    return inventory_items

In [18]:
driver = webdriver.Chrome()
driver.get("https://ironwoodrpg.com")

# Sign into the website and go to the main page
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/div[4]/button[2]' , timeout=100)
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[4]/button' , timeout=100)
wait_and_type_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[1]/input', loginInfo.usernamelist[0], timeout=100)
wait_and_type_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[3]/input', loginInfo.passwordlist[0], timeout=100)
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/button' , timeout=100)


In [28]:
find_inventory_elements(driver)

[{'name': 'wood-spruce',
  'xpath': "(//button[contains(@class, 'item') and contains(@class, 'ng-star-inserted')])[1]",
  'quantity': '44.8K'},
 {'name': 'wood-pine',
  'xpath': "(//button[contains(@class, 'item') and contains(@class, 'ng-star-inserted')])[2]",
  'quantity': '103.8K'},
 {'name': 'rock-copper',
  'xpath': "(//button[contains(@class, 'item') and contains(@class, 'ng-star-inserted')])[3]",
  'quantity': '8.9K'},
 {'name': 'gem-ruby',
  'xpath': "(//button[contains(@class, 'item') and contains(@class, 'ng-star-inserted')])[4]",
  'quantity': '99'},
 {'name': 'food-grapes',
  'xpath': "(//button[contains(@class, 'item') and contains(@class, 'ng-star-inserted')])[5]",
  'quantity': '2.3K'},
 {'name': 'food-apple',
  'xpath': "(//button[contains(@class, 'item') and contains(@class, 'ng-star-inserted')])[6]",
  'quantity': '5.3K'},
 {'name': 'challenge-scroll',
  'xpath': "(//button[contains(@class, 'item') and contains(@class, 'ng-star-inserted')])[7]",
  'quantity': '19'}]

In [13]:
import subprocess

def kill_chrome():
    # run the 'pkill' command to kill all 'chrome' processes
    subprocess.run(['pkill', 'Google Chrome'])

kill_chrome()